In [1]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from lstm_autoencoder_source import CNN_Autoencoder
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import cv2
import pickle
import mediapipe as mp

In [7]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('mps') if torch.mps.is_available() else torch.device('cpu')
squat_model_file_1 = 'model/squat_train_l1.pt'
model_squat_1 = CNN_Autoencoder().to(device)
model_squat_1.load_state_dict(torch.load(squat_model_file_1,map_location=device))

C:\Users\JPJ\AppData\Local\Temp\ipykernel_7692\734849781.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_squat_1.load_state_dict(torch.load(squat_model_file_1,map_

RuntimeError: Error(s) in loading state_dict for CNN_Autoencoder:
	Missing key(s) in state_dict: "encoder.3.weight", "encoder.3.bias". 
	Unexpected key(s) in state_dict: "encoder.2.weight", "encoder.2.bias", "encoder.4.weight", "encoder.4.bias", "decoder.4.weight", "decoder.4.bias". 
	size mismatch for decoder.0.weight: copying a param with shape torch.Size([64, 32, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 16, 3, 3]).
	size mismatch for decoder.0.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for decoder.2.weight: copying a param with shape torch.Size([32, 16, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 1, 3, 3]).
	size mismatch for decoder.2.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([1]).

In [8]:
# Mediapipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# 웹캠 열기
cap = cv2.VideoCapture('squat/squat_001.mp4')
# 해상도 설정 (가로, 세로)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 가로 해상도 설정
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # 세로 해상도 설정

# 프레임 속도 설정
cap.set(cv2.CAP_PROP_FPS, 30)  # 30 FPS로 설정
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # Mediapipe는 RGB 이미지를 처리하므로 변환
    frame = cv2.resize(frame,(640,480))
    frame2 = frame.copy()
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)
    # 관절 연결 및 포인트 그리기

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
        )
        mp_drawing.draw_landmarks(
            frame2, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
        )
    input_tensor = torch.tensor(black_frame[:, :, 1], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    
    # 모델 추론
    reconstructed_frame_1 = model_squat_1(input_tensor)
    
    # 조건부 연산: 0.5보다 크면 255, 아니면 0
    reconstructed_frame_1 = (reconstructed_frame_1 ==1).int() * 255
  
    # 모델의 출력 변환
    output_frame_1 = reconstructed_frame_1.squeeze().cpu().detach().numpy().astype(np.uint8)
    # output_frame을 3채널로 변환 (BGR)
    output_frame_color_1 = cv2.cvtColor(output_frame_1, cv2.COLOR_GRAY2BGR)


    # output_frame_color의 데이터 타입을 frame과 맞춤 (uint8)
    output_frame_color_1 = output_frame_color_1.astype(np.uint8)
    # 웹캠 화면과 합성
    output_frame_color_1 = np.zeros((output_frame_1.shape[0], output_frame_1.shape[1], 3), dtype=np.uint8)
    output_frame_color_1[:, :, 1] = output_frame_1  # 초록 채널만 활성화



    blended_frame_2 = cv2.addWeighted(frame2, 0.7, output_frame_color_1, 1.3, 0)
    # 결과를 화면에 표시
    #cv2.imshow("reality_1", blended_frame_1)
    cv2.imshow("before",blended_frame_2)
    # 종료 조건 (q 키)


    if cv2.waitKey(1) == ord('q'):
        break

# 리소스 해제
cap.release()
cv2.destroyAllWindows()

cv2.waitKey(1)

웹캠에서 프레임을 읽을 수 없습니다.


-1